# 한글 댓글 LDA

In [2]:
import re, time, requests, json
import numpy as np
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
okt = Okt()

In [3]:
df = pd.read_csv('C:/Users/twentystones/Desktop/사회연결망분석텀프로젝트/blog_crawling.csv', encoding='utf-8-sig', header=None)
df.rename(columns= {0:'title', 1:'URL', 2:'day', 3:'body'}, inplace=True)
df = df.iloc[:,3:]
df = df.fillna('')
df

,body
0,청담 파인다이닝 도쿄등심에서 이렇게 한우를 드세요도쿄등심 청담점에 다녀왔습니다.누네...
1,날씨도 추워졌겠다 오랜만에 너덜너덜해진 열손가락의 꽃단장을 위하여 덕천동 네일로 향...
2,"안녕하세요, 도쿄드라이브입니다. 최근 롤렉스나 에르메스의 인기모델들은 정식매장에 가..."
3,2021.11.17 아침 사진이 하나뚜 없ㄴㅔ건강보험 뭐 우편 온거 땜에 일어나서 ...
4,곧 다가올 와이프님과의 5주년 결혼기념일.​포근한 주말을 맞이하여 분당에서 광교앨리...
5,와 나 300일이나 여기에 살았네..3일째랑 30일째 되는 날엔 뭘 했지...?기억...
6,선릉역 소고기 맛집 도쿄등심선릉점 와이프와 함께 코엑스 전시보고 맛있는 소고기 먹으...
7,"작은도쿄평일 11:30~21:00브레이크타임 15:00~17:00토일,공휴일 휴무숙..."
8,도곡동 맛집 분위기 끝판왕 도쿄등심 도곡점​​​ ​​* 안녕하세요 제이드입니다. ...
9,여의도는 수많은 빌딩들이 밀집되어 있어화려한 건축물의 조명등으로 야경이 아름다운 ...


In [5]:
# 명사 추출
comment_okt_nouns = []
for i in tqdm(range(len(df))):
    comment_okt_nouns.append(okt.nouns(df.body[i]))
df['comment_okt_nouns'] = comment_okt_nouns 

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:04<00:00,  6.32it/s]


In [7]:
#한글자짜리 지우기
comment_okt_nouns_len = []
for i in range(len(df)):
    comment_okt_nouns_len.append([])

for i in range(len(comment_okt_nouns)):
    for word in comment_okt_nouns[i]:
        if len(word)!=1:
            comment_okt_nouns_len[i].append(word)

In [15]:
# 지칭 불용어 제거
stop_words_noun = ['사람','다음','정도', '하나', '이거', '여기', '이번', '인간', '어디','겁니다','이게','저거','이건','걸로','그거','그걸','그게','저기','그것','이것','저것','무엇','어디']
comment_okt_nouns_len_stp = []
for i in range(len(df)):
    comment_okt_nouns_len_stp.append([])

for i in range(len(df)):
    for word in comment_okt_nouns_len[i]:
        if word not in stop_words_noun:
            comment_okt_nouns_len_stp[i].append(word)
df['comment_okt_nouns_len_stp'] = comment_okt_nouns_len_stp

In [16]:
df

,body,comment_okt_nouns,comment_okt_nouns_len_stp
0,청담 파인다이닝 도쿄등심에서 이렇게 한우를 드세요도쿄등심 청담점에 다녀왔습니다.누네...,"[청담, 다이닝, 도쿄, 등심, 한우, 도쿄, 등심, 청담, 점, 누, 안과, 신랑...","[청담, 다이닝, 도쿄, 등심, 한우, 도쿄, 등심, 청담, 안과, 신랑, 라섹, ..."
1,날씨도 추워졌겠다 오랜만에 너덜너덜해진 열손가락의 꽃단장을 위하여 덕천동 네일로 향...,"[날씨, 만, 손가락, 꽃, 단장, 위, 덕천동, 네일, 향, 제, 성격, 좀, 무...","[날씨, 손가락, 단장, 덕천동, 네일, 성격, 무딘편, 변화, 대해, 크게, 신경..."
2,"안녕하세요, 도쿄드라이브입니다. 최근 롤렉스나 에르메스의 인기모델들은 정식매장에 가...","[도쿄, 드라이브, 최근, 롤렉스, 에르메스, 인기, 모델, 정식, 매장, 가면, ...","[도쿄, 드라이브, 최근, 롤렉스, 에르메스, 인기, 모델, 정식, 매장, 가면, ..."
3,2021.11.17 아침 사진이 하나뚜 없ㄴㅔ건강보험 뭐 우편 온거 땜에 일어나서 ...,"[아침, 사진, 하나, 뚜, 건강, 보험, 뭐, 우편, 땜, 일어나서, 고구마, 고...","[아침, 사진, 건강, 보험, 우편, 일어나서, 고구마, 고민, 기어, 사실, 약간..."
4,곧 다가올 와이프님과의 5주년 결혼기념일.​포근한 주말을 맞이하여 분당에서 광교앨리...,"[곧, 와이프, 주년, 결혼기념일, 주말, 맞이, 분당, 광교, 앨리, 웨이, 데이...","[와이프, 주년, 결혼기념일, 주말, 맞이, 분당, 광교, 앨리, 웨이, 데이트, ..."
5,와 나 300일이나 여기에 살았네..3일째랑 30일째 되는 날엔 뭘 했지...?기억...,"[나, 여기, 날, 뭘, 기억, 힝, 이제, 집, 전철, 때, 쭉, 신주쿠, 히가시...","[기억, 이제, 전철, 신주쿠, 히가시구, 치쪽, 해도, 진단, 오늘, 오픈, 멤버..."
6,선릉역 소고기 맛집 도쿄등심선릉점 와이프와 함께 코엑스 전시보고 맛있는 소고기 먹으...,"[선릉역, 소고기, 맛집, 도쿄, 등심, 선릉, 점, 와이프, 코엑스, 전시, 보고...","[선릉역, 소고기, 맛집, 도쿄, 등심, 선릉, 와이프, 코엑스, 전시, 보고, 소..."
7,"작은도쿄평일 11:30~21:00브레이크타임 15:00~17:00토일,공휴일 휴무숙...","[도쿄, 평일, 브레이크, 타임, 토일, 공휴일, 휴무, 숙대입구역, 근처, 늘, ...","[도쿄, 평일, 브레이크, 타임, 토일, 공휴일, 휴무, 숙대입구역, 근처, 웨이팅..."
8,도곡동 맛집 분위기 끝판왕 도쿄등심 도곡점​​​ ​​* 안녕하세요 제이드입니다. ...,"[도곡동, 맛집, 분위기, 끝판, 도쿄, 등심, 도곡, 점, 제이드, 지난주, 저녁...","[도곡동, 맛집, 분위기, 끝판, 도쿄, 등심, 도곡, 제이드, 지난주, 저녁, 약..."
9,여의도는 수많은 빌딩들이 밀집되어 있어화려한 건축물의 조명등으로 야경이 아름다운 ...,"[여의도, 빌딩, 밀집, 건축물, 조명등, 야경, 서울, 대표, 오피스, 상권, 더...","[여의도, 빌딩, 밀집, 건축물, 조명등, 야경, 서울, 대표, 오피스, 상권, 도..."


In [12]:
# 특수문자 제거
#df.comment_okt_nouns_len_stp = df.comment_okt_nouns_len_stp.str.replace(pat=r'[^\w]', repl=r' ', regex=True)

In [17]:
# 숫자 제거
#df.comment_okt_nouns_len_stp = df.comment_okt_nouns_len_stp.str.replace(r'\d+','')
#comment_okt_nouns_len_stp = df.comment_okt_nouns_len_stp

In [26]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [28]:
#bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
#trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

NameError: name 'bigram' is not defined

In [18]:
# bigram 모델 생성
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

data_words_bigrams = make_bigrams(comment_okt_nouns_len_stp)
id2word = corpora.Dictionary(data_words_bigrams)
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]

NameError: name 'bigram_mod' is not defined

In [29]:
#LDA
from gensim.models.wrappers import LdaMallet
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=30, id2word=id2word)

ModuleNotFoundError: No module named 'gensim.models.wrappers'

In [ ]:
# coherence 평가
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
  
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

#k = 5,10,15,20 한 결과 k = 10 선택
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_words_bigrams, start=5, limit=25, step=5)

In [ ]:
# document 별 topic 확률 확인하는 dataframe 만들기
idx,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9=[],[],[],[],[],[],[],[],[],[],[]
for i in range(len(corpus)):
  document_topics = gensimmodel10.get_document_topics(corpus[i])
  idx.append(i)
  topic0.append(document_topics[0][1])
  topic1.append(document_topics[1][1])
  topic2.append(document_topics[2][1])
  topic3.append(document_topics[3][1])
  topic4.append(document_topics[4][1])
  topic5.append(document_topics[5][1])
  topic6.append(document_topics[6][1])
  topic7.append(document_topics[7][1])
  topic8.append(document_topics[8][1])
  topic9.append(document_topics[9][1])
df_topic_probs_df['idx'] = idx
df_topic_probs_df['topic0'] = topic0
df_topic_probs_df['topic1'] = topic1
df_topic_probs_df['topic2'] = topic2
df_topic_probs_df['topic3'] = topic3
df_topic_probs_df['topic4'] = topic4
df_topic_probs_df['topic5'] = topic5
df_topic_probs_df['topic6'] = topic6
df_topic_probs_df['topic7'] = topic7
df_topic_probs_df['topic8'] = topic8
df_topic_probs_df['topic9'] = topic9

In [ ]:
# document별로 topic 확률을 토대로 속성 확률 구하기
df_topic_probs_df['att1'] = (df_topic_probs_df['topic0']+ df_topic_probs_df['topic1']+ df_topic_probs_df['topic3']+ df_topic_probs_df['topic6'] + df_topic_probs_df['topic9'])/5
df_topic_probs_df['att2'] = (df_topic_probs_df['topic2']+df_topic_probs_df['topic5']+df_topic_probs_df['topic7'])/3
df_topic_probs_df['att3'] = (df_topic_probs_df['topic4']+df_topic_probs_df['topic8'])/2

In [ ]:
df_topic_probs_df = df_topic_probs_df[['att1','att2','att3']]
df_topic_probs_df_argmax = []
for i in range(len(df_topic_probs_df)):
  df_topic_probs_df_argmax.append(np.argmax(df_topic_probs_df.loc[i]))
df_topic_probs_df['argmax'] = df_topic_probs_df['argmax'] +1

In [ ]:
df['attribute_num'] = df_topic_probs_df.argmax

# 영어 댓글 LDA

In [ ]:
df = pd.read_csv('/content/gdrive/Shareddrives/데청캠/웹툰 세계화/데이터 모음집/ew_2_comment.csv')
df = df[['title_a','ep_title_a','num_list','best_comment']]
for i in range(len(df)):
  if df['best_comment'][i][:3]=='TOP':
    df['best_comment'][i] = df['best_comment'][i][3:]

In [ ]:
# 명사 tokenize
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

result_nouns=[]
for i in range(len(df)):
  result_nouns.append([])

for i in range(len(df)):
  txt =  df.best_comment[i]
  tokenized = nltk.word_tokenize(txt)
  for word,pos in nltk.pos_tag(tokenized):
    if pos[:2] == 'NN':
      result_nouns[i].append(word)


In [ ]:
# 불용어 제거
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords  
stop_words = set(stopwords.words('english')) 

result_nouns_stopwords=[]
for i in range(len(result_nouns)):
  result_nouns_stopwords.append([])

for i in range(len(result_nouns)):
  for word in result_nouns[i]:
    if word not in stop_words:
      result_nouns_stopwords[i].append(word) #result_nouns_stopwords
df['result_nouns_stopwords'] = result_nouns_stopwords

In [ ]:
# 소문자로 바꾸기
df.result_nouns_stopwords = df.result_nouns_stopwords.str.lower()

In [ ]:
# 특수문자 제거
df.result_nouns_stopwords = df.result_nouns_stopwords.str.replace(pat=r'[^\w]', repl=r' ', regex=True)

In [ ]:
# 숫자 제거
df.result_nouns_stopwords = df.result_nouns_stopwords.str.replace(r'\d+','')

In [ ]:
# 길이가 짧은 단어 제거
df.result_nouns_stopwords = df.result_nouns_stopwords.str.replace(r'\W*\b\w{1,2}\b','')
result_nouns_stopwords = df.result_nouns_stopwords

In [ ]:
# bigram 모델 생성
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

data_words_bigrams = make_bigrams(result_nouns_stopwords)
id2word = corpora.Dictionary(data_words_bigrams)
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
# LDA
from gensim.models.wrappers import LdaMallet
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=30, id2word=id2word)

In [ ]:
# coherence 평가
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
  
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

#k = 10,20,30,40 평가 하였고 k = 30 선택
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_words_bigrams, start=10, limit=50, step=10)

In [ ]:
# document 별 topic 확률 확인하는 dataframe 만들기
idx,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9=[],[],[],[],[],[],[],[],[],[],[]
topic10,topic11,topic12,topic13,topic14,topic15,topic16,topic17,topic18,topic19, = [],[],[],[],[],[],[],[],[],[]
topic20,topic21,topic22,topic23,topic24,topic25,topic26,topic27,topic28,topic29 = [],[],[],[],[],[],[],[],[],[]
for i in range(len(corpus)):
  document_topics = gensimmodel30.get_document_topics(corpus[i])
  idx.append(i)
  try:
    topic0.append(document_topics[0][1])
  except:
    topic0.append(0)
  try:
    topic1.append(document_topics[1][1])
  except:
    topic1.append(0)
  try:
    topic2.append(document_topics[2][1])
  except:
    topic2.append(0)
  try:
    topic3.append(document_topics[3][1])
  except:
    topic3.append(0)
  try:
    topic4.append(document_topics[4][1])
  except:
    topic4.append(0)  
  try:
    topic5.append(document_topics[5][1])
  except:
    topic5.append(0)  
  try:
    topic6.append(document_topics[6][1])
  except:
    topic6.append(0)
  try:
    topic7.append(document_topics[7][1])
  except:
    topic7.append(0)
  try:
    topic8.append(document_topics[8][1])
  except:
    topic8.append(0)
  try:
    topic9.append(document_topics[9][1])
  except:
    topic9.append(0)
  try:
    topic10.append(document_topics[10][1])
  except:
    topic10.append(0)
  try:
    topic11.append(document_topics[11][1])
  except:
    topic11.append(0)
  try:
    topic12.append(document_topics[12][1])
  except:
    topic12.append(0)
  try:
    topic13.append(document_topics[13][1])
  except:
    topic13.append(0)
  try:
    topic14.append(document_topics[14][1])
  except:
    topic14.append(0)
  try:
    topic15.append(document_topics[15][1])
  except:
    topic15.append(0)
  try:
    topic16.append(document_topics[16][1])
  except:
    topic16.append(0)
  try:
    topic17.append(document_topics[17][1])
  except:
    topic17.append(0)
  try:
    topic18.append(document_topics[18][1])
  except:
    topic18.append(0)
  try:
    topic19.append(document_topics[19][1])
  except:
    topic19.append(0)
  try:
    topic20.append(document_topics[20][1])
  except:
    topic20.append(0)
  try:
    topic21.append(document_topics[21][1])
  except:
    topic21.append(0)
  try:
    topic22.append(document_topics[22][1])
  except:
    topic22.append(0)
  try:
    topic23.append(document_topics[23][1])
  except:
    topic23.append(0)
  try:
    topic24.append(document_topics[24][1])
  except:
    topic24.append(0)
  try:
    topic25.append(document_topics[25][1])
  except:
    topic25.append(0)

  try:
    topic26.append(document_topics[26][1])
  except:
    topic26.append(0)

  try:
    topic27.append(document_topics[27][1])
  except:
    topic27.append(0)

  try:
    topic28.append(document_topics[28][1])
  except:
    topic28.append(0)
  try:
    topic29.append(document_topics[29][1])
  except:
    topic29.append(0)
df_topic_probs_df['idx'] = idx
df_topic_probs_df['topic0'] = topic0
df_topic_probs_df['topic1'] = topic1
df_topic_probs_df['topic2'] = topic2
df_topic_probs_df['topic3'] = topic3
df_topic_probs_df['topic4'] = topic4
df_topic_probs_df['topic5'] = topic5
df_topic_probs_df['topic6'] = topic6
df_topic_probs_df['topic7'] = topic7
df_topic_probs_df['topic8'] = topic8
df_topic_probs_df['topic9'] = topic9
df_topic_probs_df['topic10'] = topic10
df_topic_probs_df['topic11'] = topic11
df_topic_probs_df['topic12'] = topic12
df_topic_probs_df['topic13'] = topic13
df_topic_probs_df['topic14'] = topic14
df_topic_probs_df['topic15'] = topic15
df_topic_probs_df['topic16'] = topic16
df_topic_probs_df['topic17'] = topic17
df_topic_probs_df['topic18'] = topic18
df_topic_probs_df['topic19'] = topic19
df_topic_probs_df['topic20'] = topic20
df_topic_probs_df['topic21'] = topic21
df_topic_probs_df['topic22'] = topic22
df_topic_probs_df['topic23'] = topic23
df_topic_probs_df['topic24'] = topic24
df_topic_probs_df['topic25'] = topic25
df_topic_probs_df['topic26'] = topic26
df_topic_probs_df['topic27'] = topic27
df_topic_probs_df['topic28'] = topic28
df_topic_probs_df['topic29'] = topic29


In [ ]:
#document 별 topic 확률을 토대로 속성 확률 구하기
df_topic_probs_df['att1'] = (df_topic_probs_df['topic1'] + df_topic_probs_df['topic2'] + df_topic_probs_df['topic3'] + df_topic_probs_df['topic4'] + df_topic_probs_df['topic5'] + df_topic_probs_df['topic6'] + df_topic_probs_df['topic7'] + df_topic_probs_df['topic8'] + df_topic_probs_df['topic9'] + df_topic_probs_df['topic11'] + df_topic_probs_df['topic12'] + df_topic_probs_df['topic13'] + df_topic_probs_df['topic14'] + + df_topic_probs_df['topic15']+ df_topic_probs_df['topic17']+ df_topic_probs_df['topic18']+ df_topic_probs_df['topic19']+ df_topic_probs_df['topic20']+ df_topic_probs_df['topic21']+ df_topic_probs_df['topic22']+ df_topic_probs_df['topic24']+ df_topic_probs_df['topic25']+ df_topic_probs_df['topic26']+ df_topic_probs_df['topic29']+df_topic_probs_df['topic28'])/25
df_topic_probs_df['att2'] = (df_topic_probs_df['topic10']+df_topic_probs_df['topic16']+df_topic_probs_df['topic23'])/3
df_topic_probs_df['att3'] = (df_topic_probs_df['topic0']+df_topic_probs_df['topic27'])/2

df_topic_probs_df = df_topic_probs_df[['att1','att2','att3']]
df_topic_probs_df_argmax = []
for i in range(len(df_topic_probs_df)):
  df_topic_probs_df_argmax.append(np.argmax(df_topic_probs_df.loc[i]))
df_topic_probs_df['argmax'] = df_topic_probs_df['argmax'] +1
df['attribute_num2'] = df_topic_probs_df.argmax

# 한글 댓글 SNA 

In [ ]:
import re
import networkx as nx
from collections import Counter


In [ ]:
num_top_nouns = 70
comment_nouns_counter = Counter(comment_mecab_nouns_len_stp_fin)
comment_top_nouns = dict(comment_nouns_counter.most_common(num_top_nouns))

In [ ]:
comment_word2id = {w: i for i, w in enumerate(comment_top_nouns.keys())}
comment_id2word = {i: w for i, w in enumerate(comment_top_nouns.keys())}

In [ ]:
import numpy as np
comment_adjacent_matrix = np.zeros((num_top_nouns, num_top_nouns), int)
for sentence in comment_mecab_nouns_len_stp_fin:
    for wi, i in comment_word2id.items():
        if wi in sentence:
            for wj, j in comment_word2id.items():
                if i != j and wj in sentence:
                    comment_adjacent_matrix[i][j] += 1
comment_adjacent_matrix

In [ ]:
comment_network = nx.from_numpy_matrix(comment_adjacent_matrix)

In [ ]:
# 폰트 확인
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
path='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname=path,size=10).get_name()
print(font_name)
plt.rc('font',family=font_name)
fm._rebuild()

In [ ]:
option = {
    'node_color' : 'lightblue',
    'node_size' : 2000
}
plt.figure(figsize=(20,20))
nx.draw_spring(comment_network, labels=comment_id2word, font_family='NanumBarunGothic', **option)